In [589]:
import keras
import numpy as np
import lstm

In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import Callable, Tuple
from keras.src.backend import Variable
import keras

class LSTMScratch():
    def __init__(
            self,
            units: int, # neuron
            keras_weight: list[Variable],
            activation: Callable[[NDArray[np.float64]], NDArray[np.float64]] = np.tanh, # activation function
    ):
        # self.input_feature = input_feature
        self.units = units
        self.kernel, self.recurrent_kernel, self.bias = keras_weight
        self.h_t = []
        self.c_t = []
        self.activation = activation

    def fit(self, input_feature: NDArray[np.float64]):
        self.input_feature = input_feature

    def predict(self, input_feature):
        sample = input_feature.shape[0]
        timestep = input_feature.shape[1]
        for i in range(sample):
            current_sample = input_feature[i]
            self.temp_h = [np.zeros(self.units)]
            self.temp_c = [np.zeros(self.units)]

            for j in range(timestep):
                h_t, c_t = self.calc(current_sample[j]) 
                self.temp_h.append(h_t)
                self.temp_c.append(c_t)
                
            self.h_t.append(self.temp_h[-1])
            self.c_t.append(self.temp_c[-1])
        return self.h_t

    def calc(self, x_t: NDArray[np.float64]):
        w_dot_x = x_t @ self.kernel
        u_dot_h = self.temp_h[-1] @ self.recurrent_kernel
        result = w_dot_x + u_dot_h + self.bias 
        k_i, k_f, k_c, k_o = np.split(result, 4, axis=0)

        i_t, f_t, o_t = self.sigmoid(k_i), self.sigmoid(k_f), self.sigmoid(k_o)

        c_tilde = self.activation(k_c)
        c_t = f_t * self.temp_c[-1] + i_t * c_tilde
        
        h_t = o_t * self.activation(c_t)

        return h_t, c_t     

    def sigmoid(self, x):
        return 1/(1+np.exp(-x))


In [591]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(5))

In [592]:
X = np.random.rand(100, 10, 2)
y = np.random.rand(100, 1)

In [593]:
model.compile(optimizer="adam", loss="mse")
model.fit(X, y, epochs=1, batch_size=16)
pred = model.predict(X[:5])
print(pred)

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.2729
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
[[-0.07914722  0.10596047  0.15951104 -0.05299308  0.25708398]
 [-0.10937162  0.09978107  0.07487479 -0.10140178  0.21825519]
 [-0.01154017  0.09046099  0.2632015   0.02523003  0.24131078]
 [-0.05898477  0.10594369  0.1677576  -0.01779553  0.2604927 ]
 [-0.09300096  0.12508826  0.01759547 -0.06021696  0.23288158]]


In [594]:
# model.summary()
# model.layers
# for i in model.layers:
#     for j in i.weights:
#         print(type(j))

In [595]:
weight = model.get_weights()

In [596]:
# arr = np.array([[1,2], [3,4], [9, 9]])
x = LSTMScratch(5, weight)

In [597]:
x.predict(X[:5])

[array([-0.07914724,  0.10596048,  0.15951111, -0.05299309,  0.25708405]),
 array([-0.10937164,  0.09978109,  0.07487484, -0.1014018 ,  0.21825527]),
 array([-0.01154016,  0.09046099,  0.26320151,  0.02523003,  0.24131081]),
 array([-0.05898479,  0.10594371,  0.16775764, -0.01779553,  0.26049276]),
 array([-0.09300098,  0.1250883 ,  0.01759547, -0.06021696,  0.23288164])]